In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

#### Run all operators

In [3]:
analysis_date = dt.date(2022, 5, 4) ##for exec board (prior run for comparison)

In [4]:
day = str(analysis_date.day).zfill(2)
month = str(analysis_date.month).zfill(2)

In [5]:
fs_list = fs.ls(f'{shared_utils.rt_utils.GCS_FILE_PATH}rt_trips/')

In [6]:
## now finds ran operators on specific analysis date
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1] and path.split('rt_trips/')[1].split('_')[1] == month and path.split('rt_trips/')[1].split('_')[2][:2] == day]

In [7]:
len(ran_operators)

44

In [8]:
# ran_operators

In [16]:
pbar = tqdm()

0it [00:00, ?it/s]

In [17]:
# analysis_date = dt.date(2022, 2, 8) ##tuesday for sacRT

In [ ]:
not_ran_operators = []
for agency in air_joined.calitp_itp_id.unique():
# for agency in [208]:
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    ## https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.export_views_gcs()
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

already ran: 484
calculating for agency: 380...
getting trips...
rt failed for agency 380
vehicle positions trip ids not in schedule
already ran: 300
already ran: 183
already ran: 182
already ran: 243
already ran: 295
already ran: 170
already ran: 45
already ran: 260
already ran: 75
already ran: 4
already ran: 310
calculating for agency: 314...
getting trips...
complete for agency: 314
already ran: 247
already ran: 315
already ran: 301
calculating for agency: 290...
getting trips...


In [ ]:
# ran_operators += [273]

In [ ]:
import yaml
import pyaml

In [ ]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    # >> filter(_.itp_id == itp_id)
    >> collect()
)

In [ ]:
airtable_organizations = airtable_organizations.dropna(subset=['itp_id']) >> mutate(itp_id = _.itp_id.astype('int64'))

In [ ]:
airtable_organizations = airtable_organizations >> filter(_.itp_id.isin(ran_operators)) >> arrange(_.caltrans_district)

In [ ]:
airtable_organizations.head(3)

# Export Working Organization View

* streamline downstream analyses, to be replaced with warehouse step...

In [ ]:
airtable_organizations['analysis_date'] = analysis_date

In [ ]:
analysis_date.isoformat()

In [ ]:
airtable_organizations.to_parquet(f'./{analysis_date.isoformat()}_working_organizations.parquet')

# Build Site

In [ ]:
with open('../portfolio/sites/rt.yml') as rt_site:
    analyses_data = yaml.load(rt_site, yaml.Loader)

In [ ]:
exclude_ids = [312, 315, 246] ##rail only systems

In [ ]:
chapters_list = []
for district in airtable_organizations.caltrans_district.unique():
    if type(district) == type(None):
        continue
    chapter_dict = {}
    filtered = (airtable_organizations
                >> filter(_.caltrans_district == district)
                >> distinct(_.itp_id, _keep_all=True)
                >> filter(-_.itp_id.isin(exclude_ids))
               )
    chapter_dict['caption'] = f'District {district}'
    chapter_dict['params'] = {'district': district}
    chapter_dict['sections'] = [{'itp_id': itp_id} for itp_id in filtered.itp_id.to_list()]
    chapters_list += [chapter_dict]

In [ ]:
parts_list = [{'chapters': chapters_list}]

In [ ]:
# parts_list

In [ ]:
# analyses_data['parts']

In [ ]:
analyses_data['parts'] = parts_list

In [ ]:
output = pyaml.dump(analyses_data)

In [ ]:
with open('../portfolio/sites/test_rt.yml', 'w') as rt_site:
    rt_site.write(output)

In [ ]:
# with open('../portfolio/sites/rt.yml', 'w') as rt_site:
#     rt_site.write(output)

### Ongoing issues
* 358 Union City too many trips without shape?

#### Parameters
* 159 1+ maps don't render
* 167 map_from_metrics fails